## Library imports

In [2]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import xlsxwriter 


## Importing list of stocks

In [3]:
stocks = pd.read_csv('../data/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2980831052321,
 'week52high': 190.21,
 'week52low': 125.81,
 'week52highSplitAdjustOnly': 183.28,
 'week52lowSplitAdjustOnly': 125.8,
 'week52change': 0.43970015350985775,
 'sharesOutstanding': 16999314721,
 'float': 0,
 'avg10Volume': 93972039,
 'avg30Volume': 97945531,
 'day200MovingAvg': 164.73,
 'day50MovingAvg': 169,
 'employees': 148641,
 'ttmEPS': 6.05,
 'ttmDividendRate': 0.9036895166695563,
 'dividendYield': 0.005209740900842786,
 'nextDividendDate': '',
 'exDividendDate': '2022-02-02',
 'nextEarningsDate': '2022-04-14',
 'peRatio': 29.55115350273185,
 'beta': 1.2917320736520443,
 'maxChangePercent': 67.75881668208254,
 'year5ChangePercent': 4.306134279320799,
 'year2ChangePercent': 1.9582800813309496,
 'year1ChangePercent': 0.4302219417845947,
 'ytdChangePercent': -0.01718293846095822,
 'month6ChangePercent': 0.2322488316019755,
 'month3ChangePercent': -0.04205854960887799,
 'month1ChangePercent': 0.05004590215084562,
 'day30ChangePe

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    try :
        api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    stock,
                    data['latestPrice'],
                    data['marketCap'],
                    'N/A'
                ],
                index = my_columns
            ), ignore_index=True
        )
    except : 
        print(stock)
final_dataframe

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [9]:
 final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

/var/folders/ct/40bmsys56hx3rxs1t6q3qyx40000gn/T/ipykernel_4398/2339524184.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/ct/40bmsys56hx3rxs1t6q3qyx40000gn/T/ipykernel_4398/2339524184.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/ct/40bmsys56hx3rxs1t6q3qyx40000gn/T/ipykernel_4398/2339524184.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/ct/40bmsys56hx3rxs1t6q3qyx40000gn/T/ipykernel_4398/2339524184.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

KeyError: 'DISCA'